In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
dataset_choice = 'mnist'
epochs = 50
batch_size = 128
noise_dim = 100
learning_rate = 0.0002
save_interval = 5

print(f"dataset_choice: {dataset_choice}")
print(f"epochs: {epochs}")
print(f"batch_size: {batch_size}")
print(f"noise_dim: {noise_dim}")
print(f"learning_rate: {learning_rate}")
print(f"save_interval: {save_interval}")

In [ ]:
print('Loading and preprocessing MNIST dataset...')
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

# Reshape to include channel dimension
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

# Normalize the images to a range of [-1, 1]
train_images = (train_images - 127.5) / 127.5

# Create a tf.data.Dataset
buffer_size = 60000 # MNIST dataset size
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

print('MNIST dataset loaded and preprocessed.')
print(f"Shape of preprocessed training images: {train_images.shape}")
print(f"Batch size: {batch_size}")
print(f"Number of batches: {len(train_dataset)}")

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(noise_dim,))) # Explicitly define input layer
    model.add(layers.Dense(7 * 7 * 256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

generator = make_generator_model()
print('Generator Model Summary:')
generator.summary()

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(28, 28, 1))) # Explicitly define input layer

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1)) # No activation, output raw logits

    return model

discriminator = make_discriminator_model()
print('Discriminator Model Summary:')
discriminator.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

print('Loss functions and optimizers defined.')

In [ ]:
noise_seed = tf.random.normal([16, noise_dim]) # Seed for visualization

@tf.function
def train_step(images):
    # 1. Generate noise for the generator input.
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images using the generator.
        generated_images = generator(noise, training=True)

        # Get discriminator predictions for real and fake images.
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate generator and discriminator losses.
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calculate gradients for the generator and apply them.
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    # Calculate gradients for the discriminator and apply them.
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

print('`train_step` function defined.')

In [ ]:
output_dir = 'generated_samples'
os.makedirs(output_dir, exist_ok=True)
print(f"Created directory: {output_dir}")

def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False. This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(10, 10))

    for i in range(predictions.shape[0]):
        plt.subplot(5, 5, i+1)
        # Rescale images from [-1, 1] to [0, 1]
        plt.imshow((predictions[i, :, :, 0] * 0.5 + 0.5), cmap='gray')
        plt.axis('off')

    plt.suptitle(f'Epoch {epoch}', fontsize=16)
    plt.savefig(os.path.join(output_dir, f'mnist_epoch_{epoch:03d}.png'))
    plt.close(fig) # Close the figure to free up memory

print('`generate_and_save_images` function defined.')

In [ ]:
print('Starting GAN training...')
for epoch in range(epochs):
    gen_losses = []
    disc_losses = []

    for batch_idx, image_batch in enumerate(train_dataset):
        g_loss, d_loss = train_step(image_batch)
        gen_losses.append(g_loss.numpy())
        disc_losses.append(d_loss.numpy())

    avg_gen_loss = np.mean(gen_losses)
    avg_disc_loss = np.mean(disc_losses)

    print(f'Epoch {epoch + 1:03d}, G_loss: {avg_gen_loss:.4f}, D_loss: {avg_disc_loss:.4f}')

    # Generate and save images at specified intervals
    if (epoch + 1) == 1 or (epoch + 1) == 2 or (epoch + 1) == 3 or (epoch + 1) == 4 or (epoch + 1) == 5 or (epoch + 1) % save_interval == 0:
        generate_and_save_images(generator, epoch + 1, noise_seed)

print('GAN training complete.')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of real images from the training dataset
for real_images_batch in train_dataset.take(1):
    real_images_display = real_images_batch.numpy()

# Generate a batch of fake images
num_images_to_compare = 10 # Let's compare 10 real and 10 fake images
noise_for_comparison = tf.random.normal([num_images_to_compare, noise_dim])
generated_images_display = generator(noise_for_comparison, training=False).numpy()

# Rescale images from [-1, 1] to [0, 1] for display
real_images_display = (real_images_display * 0.5 + 0.5)
generated_images_display = (generated_images_display * 0.5 + 0.5)

plt.figure(figsize=(10, 4))
for i in range(num_images_to_compare):
    # Display Real Images
    plt.subplot(2, num_images_to_compare, i + 1)
    plt.imshow(real_images_display[i, :, :, 0], cmap='gray')
    plt.axis('off')
    if i == 0:
        plt.title('Real', fontsize=12)

    # Display Generated Images
    plt.subplot(2, num_images_to_compare, i + 1 + num_images_to_compare)
    plt.imshow(generated_images_display[i, :, :, 0], cmap='gray')
    plt.axis('off')
    if i == 0:
        plt.title('Generated', fontsize=12)

plt.suptitle('Comparison of Real and Generated MNIST Images', fontsize=16, y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
final_output_dir = 'final_generated_images'
os.makedirs(final_output_dir, exist_ok=True)
print(f"Created directory: {final_output_dir}")

num_final_images = 100
# Generate 100 random noise vectors
final_noise = tf.random.normal([num_final_images, noise_dim])

# Generate images using the trained generator
print(f'Generating {num_final_images} final images...')
final_generated_images = generator(final_noise, training=False)

# Save each generated image individually
for i in range(num_final_images):
    image = final_generated_images[i, :, :, 0] # Get the single channel image
    # Rescale images from [-1, 1] to [0, 1]
    image = (image * 0.5 + 0.5).numpy()

    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(final_output_dir, f'final_image_{i:03d}.png'))
    plt.close() # Close the figure to free up memory

print(f'{num_final_images} final images saved to {final_output_dir}/')


In [ ]:
print('Loading and preprocessing MNIST dataset for classifier...')
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshape images to (num_samples, 28, 28, 1) and normalize to [0, 1]
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print('MNIST dataset loaded and preprocessed for classifier.')
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
print('Building and compiling classifier model...')
def build_classifier():
    model = models.Sequential([
        tf.keras.Input(shape=(28, 28, 1)), # Explicitly define input layer
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax') # 10 output classes for MNIST
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

classifier = build_classifier()
print('Classifier Model Summary:')
classifier.summary()

In [ ]:
print('Training classifier model...')
classifier_epochs = 5
classifier.fit(x_train, y_train, epochs=classifier_epochs, validation_data=(x_test, y_test), verbose=1)
print(f'Classifier trained for {classifier_epochs} epochs.')

In [ ]:
print('Predicting labels for generated images...')

# Normalize generated images from [-1, 1] to [0, 1]
normalized_generated_images = (final_generated_images * 0.5 + 0.5).numpy()

# Predict labels using the trained classifier
predictions = classifier.predict(normalized_generated_images)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the distribution of predicted labels
label_distribution = {i: 0 for i in range(10)}
for label in predicted_labels:
    label_distribution[label] += 1

print('\nPredicted label distribution for generated images:')
for label, count in label_distribution.items():
    print(f'Digit {label}: {count} images')
